In [24]:
import os
import ssl
import httpx
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE
http_client = httpx.Client(verify=False)
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY2"), http_client=http_client)

In [25]:
file = client.files.create(
    file=open("sales_data.csv", "rb"),
    purpose="assistants"
)

In [26]:
print(file.id)

file-NiAp6SgTSuNupBoFHtHTTW


In [27]:
assistant =client.beta.assistants.create(
    name = "Analista de Dados",
    instructions = "Você é um analista que analisa dados sobre vendas",
    tools = [{"type": "code_interpreter"}],
    tool_resources = {"code_interpreter": {"file_ids": [file.id]}},
    model = "gpt-4o"
)

In [38]:
#pergunta = "Qual o rating média das vendas do supermercado?"
pergunta = "Gere um gráfico de pizza com o percentual de vendas por linha de produto"

In [39]:
thread = client.beta.threads.create()
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = pergunta
)

C:\Users\fmtorres\AppData\Local\Temp\ipykernel_3480\3685932091.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
C:\Users\fmtorres\AppData\Local\Temp\ipykernel_3480\3685932091.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.create(


In [40]:
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id,
    instructions = "Nome de usuário premium"
)

C:\Users\fmtorres\AppData\Local\Temp\ipykernel_3480\3264856326.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create(


In [43]:
run.status

'completed'

In [42]:
import time
while run.status in ["queued", "in_progress","cancelling"]:
    print(f"Status: {run.status}")
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id = thread.id,
        run_id = run.id
    )

Status: queued


C:\Users\fmtorres\AppData\Local\Temp\ipykernel_3480\3618006381.py:5: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.retrieve(


Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress
Status: in_progress


In [44]:
if run.status == "completed":
    mensagens = client.beta.threads.messages.list(
        thread_id = thread.id
    )
    print(mensagens)
else:
    print(f"Erro: {run.status}")

C:\Users\fmtorres\AppData\Local\Temp\ipykernel_3480\3003347538.py:2: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  mensagens = client.beta.threads.messages.list(


SyncCursorPage[Message](data=[Message(id='msg_nFT2VIKVWiFSCBt5WhD2cFlK', assistant_id='asst_qAnhhkczAu4SUlBoffbXGtqQ', attachments=[], completed_at=None, content=[ImageFileContentBlock(image_file=ImageFile(file_id='file-HuyuV7VsJashmKub5Bdk5X', detail=None), type='image_file'), TextContentBlock(text=Text(annotations=[], value='Aqui está o gráfico de pizza que mostra o percentual de vendas por linha de produto. Cada fatia representa a proporção das vendas totais atribuídas a cada linha de produto. Se precisar de mais alguma coisa, sinta-se à vontade para perguntar!'), type='text')], created_at=1767718038, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_GaTlB26dB1R4lQXQN4avJi0e', status=None, thread_id='thread_8X3eX3eXpCgKI2EF9A1GBnQP'), Message(id='msg_T860Uv2ncxB2WBrUXf6tWcKd', assistant_id='asst_qAnhhkczAu4SUlBoffbXGtqQ', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Os dad

In [45]:
print(mensagens.data[0].content[0].text.value)

AttributeError: 'ImageFileContentBlock' object has no attribute 'text'

In [46]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id = thread.id,
    run_id = run.id
)

C:\Users\fmtorres\AppData\Local\Temp\ipykernel_3480\2285723283.py:1: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run_steps = client.beta.threads.runs.steps.list(


In [50]:
for step in run_steps.data[::-1]:
    print(f"\n====Step {step.step_details.type}")
    if step.step_details.type == "tool_calls":
        for tool_call in step.step_details.tool_calls:
            print("=" *10)
            print(tool_call.code_interpreter.input)
            print("="*10)
    if step.step_details.type == "message_creation":
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )
        if message.content[0].type == "text":
            print(message.content[0].text.value)
        if message.content[0].type == "image_file":
            file_id = message.content[0].image_file.file_id
            image_data = client.files.content(file_id)
            with open(f"files_2/{file_id}.png", "wb") as f:
                f.write(image_data.read())
                print({f"Imagem {file_id} salva"})


====Step message_creation


C:\Users\fmtorres\AppData\Local\Temp\ipykernel_3480\4283117408.py:9: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  message = client.beta.threads.messages.retrieve(


Primeiro, vamos inspecionar o arquivo que você enviou para entender seu conteúdo. Isso nos ajudará a identificar as colunas relevantes para gerar o gráfico de pizza de vendas por linha de produto.

====Step tool_calls
import pandas as pd

# Carregar o arquivo para inspecionar o conteúdo
file_path = '/mnt/data/file-NiAp6SgTSuNupBoFHtHTTW'
data = pd.read_csv(file_path)

# Mostrar as primeiras linhas do dataframe
data.head()

====Step message_creation
Os dados contêm várias colunas, incluindo a linha de produto ("Product line") e o total de vendas ("Total"). Vamos gerar um gráfico de pizza que mostra o percentual de vendas por linha de produto usando esses dados.

====Step tool_calls
import matplotlib.pyplot as plt

# Agrupar os dados por linha de produto e somar as vendas totais
sales_by_product_line = data.groupby('Product line')['Total'].sum()

# Plotando o gráfico de pizza
plt.figure(figsize=(10, 7))
plt.pie(sales_by_product_line, labels=sales_by_product_line.index, autopct='%1.1f%%',